In [1]:

from decimal import Decimal
import pandas as pd
from django.db.models import F, Value, CharField, Q, Sum
from django.db.models.functions import Concat, Coalesce
from project.models import Project
from public_data.models import CommunePop, Cerema, CommuneDiff, Scot
from public_data.storages import DataStorage

project = Project.objects.get(id=1504)
project.name

'Diagnostic de CA Grand Auch Coeur de Gascogne'

In [2]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("pop_change"),
        Total=F("pop"),
    )
    .values(*headers)
)
qs[:10]

<QuerySet [{'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2019, 'Changement': 1, 'Total': 342}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2018, 'Changement': -5, 'Total': 341}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2017, 'Changement': -3, 'Total': 346}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2016, 'Changement': -3, 'Total': 349}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2015, 'Changement':

In [3]:
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                              2006   
Commune              Insee EPCI                            SCoT        Département Région                 
Antras               32003 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   

                                                                                                     \
Année                         

In [4]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("household_change"),
        Total=F("household"),
    )
    .values(*headers)
)
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                              2006   
Commune              Insee EPCI                            SCoT        Département Région                 
Antras               32003 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   

                                                                                                   \
Année                           

In [5]:
config = [
    {"name": "Insee", "db": F("city__insee"), "type": "index"},
    {"name": "Commune", "db": F("city__name"), "type": "index"},
    {"name": "EPCI", "db": F("city__epci__name"), "type": "index"},
    {"name": "SCoT", "db": F("city__scot__name"), "type": "index"},
    {
        "name": "Département",
        "db": F("city__departement__name"),
        "type": "index",
    },
    {
        "name": "Région",
        "db": F("city__departement__region__name"),
        "type": "index",
    },
    {
        "name": "Plus_ancien_millésime",
        "db": Value(project.first_year_ocsge),
        "type": "index",
    },
    {
        "name": "Plus_récent_millésime",
        "db": Value(project.last_year_ocsge),
        "type": "index",
    },
    {
        "name": "Différentiel",
        "db": Value(project.last_year_ocsge),
        "type": "columns",
    },
]
qs = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(), matrix__is_artificial=True, year=project.last_year_ocsge
    )
    .annotate(**{_["name"]: _["db"] for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(Surface_artificielle_dernier_millésime=Sum("surface"))
    .order_by("Insee")
)
df = (
    pd.DataFrame(
        qs,
        columns=[
            *[_["name"] for _ in config],
            "Surface_artificielle_dernier_millésime",
        ],
    )
    .fillna(0.0)
    .pivot(
        columns=[_["name"] for _ in config if _["type"] == "columns"],
        index=[_["name"] for _ in config if _["type"] == "index"],
        values=["Surface_artificielle_dernier_millésime"],
    )
)
df

,,,,,,,,Surface_artificielle_dernier_millésime
,,,,,,,Différentiel,2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,17.3000
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1702.3100
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,24.2800
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,93.0400
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,39.5100
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,127.8400
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,67.4800
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,157.2300


In [6]:
total_artificielle = sum(_["Surface_artificielle_dernier_millésime"] for _ in qs)
total_artificielle

Decimal('5000.6700')

In [7]:
config[8]["db"] = Concat(
    "year_old", Value("-"), "year_new", output_field=CharField()
)
qs2 = (
    CommuneDiff.objects.filter(
        city__in=project.cities.all(),
        year_old__gte=project.analyse_start_date,
        year_old__lte=project.analyse_end_date,
    )
    .annotate(
        **{_["name"]: _["db"] for _ in config},
        Artificialisation=F("new_artif"),
        Renaturation=F("new_natural"),
        Artificialisation_nette=F("net_artif"),
    )
    .values(
        *[_["name"] for _ in config],
        "Artificialisation",
        "Renaturation",
        "Artificialisation_nette",
    )
)
qs2[:3]

<QuerySet [{'Insee': '32207', 'Commune': 'Leboulin', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Plus_ancien_millésime': 2016, 'Plus_récent_millésime': 2019, 'Différentiel': '2016-2019', 'Artificialisation': Decimal('0.3900'), 'Renaturation': Decimal('0.0200'), 'Artificialisation_nette': Decimal('0.3700')}, {'Insee': '32014', 'Commune': 'Augnax', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Plus_ancien_millésime': 2016, 'Plus_récent_millésime': 2019, 'Différentiel': '2016-2019', 'Artificialisation': Decimal('1.2700'), 'Renaturation': Decimal('0.0600'), 'Artificialisation_nette': Decimal('1.2100')}, {'Insee': '32089', 'Commune': 'Castillon-Massas', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Plus_ancien_millésime': 2016, 'Plus_récent_millésime': 2019, 'Différentiel': '2016-2019', 'Art

In [8]:
df2 = (
    pd.DataFrame(
        qs2,
        columns=[
            *[_["name"] for _ in config],
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette",
        ],
    )
    .fillna(0.0)
    .pivot(
        columns=[_["name"] for _ in config if _["type"] == "columns"],
        index=[_["name"] for _ in config if _["type"] == "index"],
        values=[
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette",
        ],
    )
)
df2

,,,,,,,,Artificialisation,Renaturation,Artificialisation_nette
,,,,,,,Différentiel,2016-2019,2016-2019,2016-2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,,,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,0.0000,0.0100,-0.0100
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,25.7900,2.1500,23.6400
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.2700,0.0600,1.2100
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.2500,0.0000,1.2500
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,0.0900,0.0000,0.0900
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.2300,0.4100,0.8200
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,0.0300,0.0000,0.0300
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.1500,0.0200,1.1300


In [9]:
df4 = df.merge(df2, left_index=True, right_index=True)
df4

,,,,,,,,Surface_artificielle_dernier_millésime,Artificialisation,Renaturation,Artificialisation_nette
,,,,,,,Différentiel,2019,2016-2019,2016-2019,2016-2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,,,,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,17.3000,0.0000,0.0100,-0.0100
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1702.3100,25.7900,2.1500,23.6400
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,24.2800,1.2700,0.0600,1.2100
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,93.0400,1.2500,0.0000,1.2500
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,39.5100,0.0900,0.0000,0.0900
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,127.8400,1.2300,0.4100,0.8200
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,67.4800,0.0300,0.0000,0.0300
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,157.2300,1.1500,0.0200,1.1300


In [10]:
df3 = df.copy()
df3 = df3.astype(float)
df3

,,,,,,,,Surface_artificielle_dernier_millésime
,,,,,,,Différentiel,2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,17.30
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1702.31
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,24.28
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,93.04
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,39.51
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,127.84
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,67.48
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,157.23


In [11]:
config = [
    {"name": "Commune", "db": "city__name", "type": "index"},
    {"name": "Insee", "db": "city__insee", "type": "index"},
    {"name": "EPCI", "db": "city__epci__name", "type": "index"},
    {"name": "SCoT", "db": "city__scot__name", "type": "index"},
    {"name": "Département", "db": "city__departement__name", "type": "index"},
    {
        "name": "Région",
        "db": "city__departement__region__name",
        "type": "index",
    },
    {"name": "Année", "db": "year", "type": "columns"},
    {
        "name": "Code",
        "db": "matrix__couverture__code_prefix",
        "type": "columns",
    },
    {
        "name": "Libellé",
        "db": "matrix__couverture__label_short",
        "type": "columns",
    },
]
qs1 = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(),
        year__gte=project.analyse_start_date,
        year__lte=project.analyse_end_date
    )
    .annotate(**{_["name"]: F(_["db"]) for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(surface=Sum("surface"))
    .order_by(*[_["name"] for _ in config])
)
config[7] = {
    "name": "Code",
    "db": "matrix__usage__code_prefix",
    "type": "columns",
}
config[8] = {
    "name": "Libellé",
    "db": "matrix__usage__label_short",
    "type": "columns",
}
qs2 = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(),
        year__gte=project.analyse_start_date,
        year__lte=project.analyse_end_date,
    )
    .annotate(**{_["name"]: F(_["db"]) for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(surface=Sum("surface"))
    .order_by(*[_["name"] for _ in config])
)
qs = qs1.union(qs2)
qs.count()

1252

In [12]:
df = (
    pd.DataFrame(qs, columns=["surface"] + [_["name"] for _ in config])
    .pivot(
        columns=[_["name"] for _ in config if _["type"] == "columns"],
        index=[_["name"] for _ in config if _["type"] == "index"],
        values=["surface"],
    )
    .fillna(0.0)
)
cols = sorted(list(df.columns.values), key=lambda x: f"{x[1]}{x[2]}")
df = df[cols]
df

surface  \
Année                                                                                                2016   
Code                                                                                            CS1.1.1.1   
Libellé                                                                                      Zones bâties   
Commune              Insee EPCI                            SCoT        Département Région                   
Antras               32003 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       2.0500   
Auch                 32013 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie     334.5400   
Augnax               32014 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       2.3800   
Auterive             32019 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      12.2600   
Ayguetinte           32024 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       5.9200   
Biran                32054 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      15.7200   
Bonas                32059 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       6.1500   
Castelnau-Barbarens  32076 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      19.6800   
Castillon-Massas     32089 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       6.1700   
Castin               32091 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       7.2500   
Castéra-Verduzan     32083 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      24.2300   
Crastes              32112 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       9.3300   
Duran                32117 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      14.0200   
Jegun                32162 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      27.3300   
Lahitte              32183 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       4.2700   
Lavardens            32204 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      15.0200   
Leboulin             32207 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       6.7800   
Mirepoix             32258 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       6.2200   
Montaut-les-Créneaux 32279 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      17.1900   
Montégut             32282 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      10.2100   
Mérens               32251 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       1.9500   
Nougaroulet          32298 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      10.6900   
Ordan-Larroque       32301 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      21.5000   
Pavie                32307 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      53.5100   
Pessan               32312 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      18.1800   
Peyrusse-Massas      32316 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       3.6400   
Preignan             32331 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      19.8900   
Puycasquier          32335 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      14.0800   
Roquefort            32347 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       6.2200   
Roquelaure           32348 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie      16.4500   
Saint-Jean-Poutge    32382 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       9.6700   
Saint-Lary           32384 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie       5.8600   
Sainte-Christie      32368 CA Grand Auch Coeur de Gascogne De gas

In [13]:
list(df.columns.values)

[('surface', 2016, 'CS1.1.1.1', 'Zones bâties'),
 ('surface', 2016, 'CS1.1.1.2', 'Zones non bâties'),
 ('surface', 2016, 'CS1.1.2.1', 'Zones à matériaux minéraux'),
 ('surface', 2016, 'CS1.2.2', "Surfaces d'eau"),
 ('surface', 2016, 'CS2.1.1.1', 'Peuplement de feuillus'),
 ('surface', 2016, 'CS2.1.1.2', 'Peuplement de conifères'),
 ('surface', 2016, 'CS2.1.1.3', 'Peuplement mixte'),
 ('surface', 2016, 'CS2.1.2', 'Formations arbustives et sous-...'),
 ('surface', 2016, 'CS2.1.3', 'Autres formations ligneuses'),
 ('surface', 2016, 'CS2.2.1', 'Formations herbacées'),
 ('surface', 2016, 'US1.1', 'Agriculture'),
 ('surface', 2016, 'US1.2', 'Sylviculture'),
 ('surface', 2016, 'US1.3', 'Activités d’extraction'),
 ('surface', 2016, 'US2', 'Secondaire'),
 ('surface', 2016, 'US235', 'Production secondaire; tertiai...'),
 ('surface', 2016, 'US3', 'Tertiaire'),
 ('surface', 2016, 'US4.1.1', 'Routier'),
 ('surface', 2016, 'US4.1.2', 'Ferré'),
 ('surface', 2016, 'US4.1.3', 'Aérien'),
 ('surface', 20

In [35]:
qs = (
    CommuneDiff.objects.all()
    .filter(city__in=project.cities.all())
    .annotate(
        period=Concat("year_old", Value(" - "), "year_new", output_field=CharField()),
        name=F("city__name"),
        area=F("city__area"),
    )
    .order_by("name", "period", "year_old", "year_new")
    .values("name", "period", "year_old", "year_new", "area")
    .annotate(
        new_artif=Coalesce(Sum("new_artif"), Decimal("0")),
        new_natural=Coalesce(Sum("new_natural"), Decimal("0")),
        net_artif=Coalesce(Sum("net_artif"), Decimal("0")),
    )
)
qs

<QuerySet [{'year_old': 2016, 'year_new': 2019, 'period': '2016 - 2019', 'name': 'Antras', 'area': Decimal('6730961.0000'), 'new_artif': Decimal('0.0000'), 'new_natural': Decimal('0.0100'), 'net_artif': Decimal('-0.0100')}, {'year_old': 2016, 'year_new': 2019, 'period': '2016 - 2019', 'name': 'Auch', 'area': Decimal('73407704.5000'), 'new_artif': Decimal('25.7900'), 'new_natural': Decimal('2.1500'), 'net_artif': Decimal('23.6400')}, {'year_old': 2016, 'year_new': 2019, 'period': '2016 - 2019', 'name': 'Augnax', 'area': Decimal('3991344.5000'), 'new_artif': Decimal('1.2700'), 'new_natural': Decimal('0.0600'), 'net_artif': Decimal('1.2100')}, {'year_old': 2016, 'year_new': 2019, 'period': '2016 - 2019', 'name': 'Auterive', 'area': Decimal('10907261.5000'), 'new_artif': Decimal('1.2500'), 'new_natural': Decimal('0.0000'), 'net_artif': Decimal('1.2500')}, {'year_old': 2016, 'year_new': 2019, 'period': '2016 - 2019', 'name': 'Ayguetinte', 'area': Decimal('6281470.0000'), 'new_artif': Decima

In [36]:
df = pd.DataFrame.from_records(qs)
df

,year_old,year_new,period,name,area,new_artif,new_natural,net_artif
0,2016,2019,2016 - 2019,Antras,6730961.0000,0.0000,0.0100,-0.0100
1,2016,2019,2016 - 2019,Auch,73407704.5000,25.7900,2.1500,23.6400
2,2016,2019,2016 - 2019,Augnax,3991344.5000,1.2700,0.0600,1.2100
3,2016,2019,2016 - 2019,Auterive,10907261.5000,1.2500,0.0000,1.2500
4,2016,2019,2016 - 2019,Ayguetinte,6281470.0000,0.0900,0.0000,0.0900
5,2016,2019,2016 - 2019,Biran,36607182.5000,1.2300,0.4100,0.8200
6,2016,2019,2016 - 2019,Bonas,10145524.5000,0.0300,0.0000,0.0300
7,2016,2019,2016 - 2019,Castelnau-Barbarens,42797751.0000,1.1500,0.0200,1.1300
8,2016,2019,2016 - 2019,Castillon-Massas,9799398.0000,0.0000,0.0000,0.0000
9,2016,2019,2016 - 2019,Castin,11391706.5000,0.0000,0.0900,-0.0900


In [38]:
pivot_df = df.pivot_table(
    index='name',
    columns='period',
    values=['new_artif', 'new_natural', 'net_artif'],
    aggfunc='sum'
)
pivot_df = pivot_df.swaplevel(0, 1, axis=1).sort_index(axis=1)
pivot_df["total_net_artif"] = pivot_df.xs('net_artif', axis=1, level=1).sum(axis=1)
pivot_df["area"] = df.groupby('name')['area'].first()
pivot_df['net_artificialization_percentage'] = (pivot_df["total_net_artif"] / pivot_df['area']) * 100
pivot_df['net_artificialization_percentage'] = pivot_df['net_artificialization_percentage'].apply(lambda x: f"{x:.2f}%")
pivot_df

period               2016 - 2019                       total_net_artif  \
                       net_artif new_artif new_natural                   
name                                                                     
Antras                   -0.0100    0.0000      0.0100         -0.0100   
Auch                     23.6400   25.7900      2.1500         23.6400   
Augnax                    1.2100    1.2700      0.0600          1.2100   
Auterive                  1.2500    1.2500      0.0000          1.2500   
Ayguetinte                0.0900    0.0900      0.0000          0.0900   
Biran                     0.8200    1.2300      0.4100          0.8200   
Bonas                     0.0300    0.0300      0.0000          0.0300   
Castelnau-Barbarens       1.1300    1.1500      0.0200          1.1300   
Castillon-Massas          0.0000    0.0000      0.0000          0.0000   
Castin                   -0.0900    0.0000      0.0900         -0.0900   
Castéra-Verduzan          3.5100    3.5100      0.0000          3.5100   
Crastes                   0.5700    0.5700      0.0000          0.5700   
Duran                     0.8800    0.8800      0.0000          0.8800   
Jegun                     6.7300    6.7500      0.0200          6.7300   
Lahitte                  -0.0200    0.0000      0.0200         -0.0200   
Lavardens                 0.5900    0.5900      0.0000          0.5900   
Leboulin                  0.3700    0.3900      0.0200          0.3700   
Mirepoix                  0.4000    0.4200      0.0200          0.4000   
Montaut-les-Créneaux      0.7400    0.8200      0.0800          0.7400   
Montégut                  3.2800    3.6600      0.3800          3.2800   
Mérens                    0.0600    0.0600      0.0000          0.0600   
Nougaroulet               0.2800    0.3200      0.0400          0.2800   
Ordan-Larroque            0.8300    1.3300      0.5000          0.8300   
Pavie                     7.8900    9.7200      1.8300          7.8900   
Pessan                    0.8500    0.8600      0.0100          0.8500   
Peyrusse-Massas           1.0200    1.0200      0.0000          1.0200   
Preignan                  0.6900    0.6900      0.0000          0.6900   
Puycasquier               2.1400    2.2300      0.0900          2.1400   
Roquefort                 0.2800    0.2800      0.0000          0.2800   
Roquelaure                1.2700    1.3000      0.0300          1.2700   
Saint-Jean-Poutge         0.8500    0.8500      0.0000          0.8500   
Saint-Lary                0.5900    1.0700      0.4800          0.5900   
Sainte-Christie           0.4700    0.4700      0.0000          0.4700   
Tourrenquets              0.0000    0.0000      0.0000          0.0000   

period                         area net_artificialization_percentage  
                                                                      
name                                                                  
Antras                 6730961.0000                           -0.00%  
Auch                  73407704.5000                            0.00%  
Augnax                 3991344.5000                            0.00%  
Auterive              10907261.5000                            0.00%  
Ayguetinte             6281470.0000                            0.00%  
Biran                 36607182.5000                            0.00%  
Bonas                 10145524.5000                            0.00%  
Castelnau-Barbarens   42797751.0000                            0.00%  
Castillon-Massas       9799398.0000                            0.00%  
Castin                11391706.5000                           -0.00%  
Castéra-Verduzan      20121813.5000                            0.00%  
Crastes               19497325.0000                            0.00%  
Duran                  6604670.0000                            0.00%  
Jegun                 39179829.0000                            0.00%  
Lahitte                5077742.0000                   

In [39]:
records = pivot_df.reset_index().to_dict(orient='records')
records

[{('name', ''): 'Antras',
  ('2016 - 2019', 'net_artif'): Decimal('-0.0100'),
  ('2016 - 2019', 'new_artif'): Decimal('0.0000'),
  ('2016 - 2019', 'new_natural'): Decimal('0.0100'),
  ('total_net_artif', ''): Decimal('-0.0100'),
  ('area', ''): Decimal('6730961.0000'),
  ('net_artificialization_percentage', ''): '-0.00%'},
 {('name', ''): 'Auch',
  ('2016 - 2019', 'net_artif'): Decimal('23.6400'),
  ('2016 - 2019', 'new_artif'): Decimal('25.7900'),
  ('2016 - 2019', 'new_natural'): Decimal('2.1500'),
  ('total_net_artif', ''): Decimal('23.6400'),
  ('area', ''): Decimal('73407704.5000'),
  ('net_artificialization_percentage', ''): '0.00%'},
 {('name', ''): 'Augnax',
  ('2016 - 2019', 'net_artif'): Decimal('1.2100'),
  ('2016 - 2019', 'new_artif'): Decimal('1.2700'),
  ('2016 - 2019', 'new_natural'): Decimal('0.0600'),
  ('total_net_artif', ''): Decimal('1.2100'),
  ('area', ''): Decimal('3991344.5000'),
  ('net_artificialization_percentage', ''): '0.00%'},
 {('name', ''): 'Auterive',
 

In [40]:
periods = pivot_df.columns.levels[0]
periods

Index(['2016 - 2019', 'total_net_artif', 'area',
       'net_artificialization_percentage'],
      dtype='object', name='period')